In [1]:
from shapely.geometry.polygon import Polygon,Point
from shapely.geometry import MultiLineString
from shapely import points,linestrings,polygons,minimum_bounding_circle,union
from shapely import GeometryCollection, LineString, MultiPoint, Point, Polygon
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeat
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import geopandas as gpd
import shapely as shp
import xmltodict
import json
import os
from datetime import datetime
import glob
from sklearn.metrics.pairwise import haversine_distances
from scipy.ndimage import gaussian_filter
import seaborn as sns
from pyproj import CRS
import xesmf as xe

In [86]:
# MEB 23 Feb 2026
# try to convert Nadocast to polygons for verf

areaempty=shp.Polygon()

ds_mask=xr.open_dataset('/users/baldwin/data/CONUS_nadocast.nc')
conus_grid=ds_mask.coarsen(y=3,x=3,boundary='pad').mean().CONUS.values
xx_mlp=np.arange(76)
yy_mlp=np.arange(129)
Xhw,Yhw=np.meshgrid(xx_mlp,yy_mlp)
X1dhw=Xhw.flatten()
Y1dhw=Yhw.flatten()
lat2d=ds_mask.coarsen(y=3,x=3,boundary='pad').mean().latitude.values
lon2d=ds_mask.coarsen(y=3,x=3,boundary='pad').mean().longitude.values-360.0
df_hwgrd=pd.DataFrame({'col':X1dhw,'row':Y1dhw,'conus':conus_grid.flatten(),'lon':lon2d.flatten(),'lat':lat2d.flatten()})
hwgrdlatlon = gpd.GeoDataFrame(df_hwgrd, geometry=gpd.points_from_xy(df_hwgrd.lon, df_hwgrd.lat)).set_crs(epsg=4326)
hwgrdproj=hwgrdlatlon.to_crs(crs=CRS.from_proj4("+proj=lcc +lat_1=25 +lat_2=25 +lon_0=-95"))
hwgrdproj['geometry']=hwgrdproj.buffer(22855.5,cap_style='square').set_precision(1.)
hwgrdproj


#DATES = pd.date_range(start='2025-10-28 12:00:00',end='2025-10-29 12:00:00',inclusive='left', freq='1D')
DATES = pd.date_range(start='2025-01-01 12:00:00',end='2025-11-01 12:00:00',inclusive='left', freq='1D')


ranges=['day1_0000']


flds=['torn','hail','wind']



for fld in flds:
    print(fld)

    
    if fld=='torn':
        #lvls=[0,2,5,10,15,30,45,60,100]
        #lvls=[0.,1.e-4,1.e-3,.01,.02,.05,.10,.15,.20,.25,.30,.45,.6,1.]
        lvls=np.array([1.e-4,1.e-3,.01,.02,.05,.10,.15,.20,.25,.30,.45,.6,1.])
        nlvls=12
        fname='nadocast_2022_models_conus_tornado'
    elif fld=='hail':
        #lvls=[0,5,15,30,45,60,100]
        #lvls=[0.,1.e-4,1.e-3,.01,.02,.05,.10,.15,.20,.25,.30,.45,.6,1.]
        lvls=np.array([1.e-4,1.e-3,.01,.02,.05,.10,.15,.20,.25,.30,.45,.6,1.])
        nlvls=12
        fname='nadocast_2022_models_conus_hail'
    elif fld=='wind':
        #lvls=[0,5,15,30,45,60,100]
        #lvls=[0.,1.e-4,1.e-3,.01,.02,.05,.10,.15,.20,.25,.30,.45,.6,1.]
        lvls=np.array([1.e-4,1.e-3,.01,.02,.05,.10,.15,.20,.25,.30,.45,.6,1.])
        nlvls=12
        fname='nadocast_2022_models_conus_wind'
    id_lvl=[]
    title_lvl=[]
    geo_lvl=[]
    for ii in np.arange(nlvls):
        lvl=lvls[ii]
        id_lvl.append(ii)
        title_lvl.append(str(lvl*100.))
        geo_lvl.append(shp.Polygon())


        
    for date in DATES:
        print(date,datetime.now())
        for range in ranges:
 
            ffld=f'{fld}_{range}'

    
            day1=date.strftime('%Y%m%d')        
            yy=date.strftime('%Y')  
            valid_time=date
            expire_time=date+pd.to_timedelta('1 day')
            issue_time=date-pd.to_timedelta('12 hours')
            valid_lvl=[]
            expire_lvl=[]
            for ii in np.arange(nlvls):
                valid_lvl.append(valid_time)
                expire_lvl.append(expire_time)
        
            df1=pd.DataFrame({'id':id_lvl,'validend':expire_time,'validstart':valid_lvl,'title':title_lvl})
            prodproj = gpd.GeoDataFrame(df1, geometry=geo_lvl).set_crs(crs=CRS.from_proj4("+proj=lcc +lat_1=25 +lat_2=25 +lon_0=-95"))

            try:
                gribfile1=f'/spcarch4/model/nadocast_grib2/{day1}/{fname}_{day1}_t00z_f12-35.grib2'
                ds_mlp=xr.open_dataset(gribfile1,engine='cfgrib', backend_kwargs={'indexpath':''})
                
                if fld=='hail':
                    hwgrdproj['mlp']=ds_mlp.coarsen(y=3,x=3,boundary='pad').mean().hailprob.values.flatten()*0.01
                if fld=='wind':
                    hwgrdproj['mlp']=ds_mlp.coarsen(y=3,x=3,boundary='pad').mean().windprob.values.flatten()*0.01
                if fld=='torn':
                    hwgrdproj['mlp']=ds_mlp.coarsen(y=3,x=3,boundary='pad').mean().torprob.values.flatten()*0.01
            except:
                #print(f'/NAWIPS/archive/OUTLOOK/{yy}/geojson/{ffld}_{day1}*.lyr.geojson file not found')
                print(gribfile1,'some kind of error')

                continue

            for ii in np.arange(nlvls):
                lvl=lvls[ii]
                lvl2=lvls[ii+1]
                try:
                    fcstarea=shp.unary_union(hwgrdproj[(hwgrdproj.mlp>=lvl)&(hwgrdproj.mlp<lvl2)].geometry).buffer(10000.).buffer(-10000.).simplify(tolerance=1000., preserve_topology=True)
                except:
                    fcstarea=areaempty
                prodproj.loc[ii,'geometry']=fcstarea
            prodproj.to_file(f'/scratch/baldwin/nadocast/{fname}_{day1}_t00z_f12-35.geojson',driver='GeoJSON')



torn
2025-01-01 12:00:00 2026-02-23 16:52:35.571688
2025-01-02 12:00:00 2026-02-23 16:52:35.812009
2025-01-03 12:00:00 2026-02-23 16:52:36.014298
2025-01-04 12:00:00 2026-02-23 16:52:36.324768
2025-01-05 12:00:00 2026-02-23 16:52:36.572759
2025-01-06 12:00:00 2026-02-23 16:52:36.868529
2025-01-07 12:00:00 2026-02-23 16:52:37.162834
2025-01-08 12:00:00 2026-02-23 16:52:37.357987
2025-01-09 12:00:00 2026-02-23 16:52:37.616966
2025-01-10 12:00:00 2026-02-23 16:52:37.900613
2025-01-11 12:00:00 2026-02-23 16:52:38.172804
2025-01-12 12:00:00 2026-02-23 16:52:38.385009
2025-01-13 12:00:00 2026-02-23 16:52:38.577975
2025-01-14 12:00:00 2026-02-23 16:52:38.756115
2025-01-15 12:00:00 2026-02-23 16:52:38.961594
2025-01-16 12:00:00 2026-02-23 16:52:39.178484
2025-01-17 12:00:00 2026-02-23 16:52:39.391582
2025-01-18 12:00:00 2026-02-23 16:52:39.664415
2025-01-19 12:00:00 2026-02-23 16:52:40.032701
2025-01-20 12:00:00 2026-02-23 16:52:40.397806
2025-01-21 12:00:00 2026-02-23 16:52:40.705171
2025-01-